In [ ]:
import pyrealsense2 as rs
import numpy as np
import open3d as o3d


In [ ]:
import pyrealsense2 as rs
import numpy as np
import open3d as o3d

# Funkcja do przetwarzania klatki i konwersji na chmurę punktów
def frame_to_point_cloud(depth_frame):
    pc = rs.pointcloud()
    print(type(pc))
    pc.map_to(frames.get_color_frame())
    points = pc.calculate(depth_frame)
    vtx = np.asanyarray(points.get_vertices())
    vtx_np = np.array([(v[0], v[1], v[2]) for v in vtx], dtype=np.float64)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(vtx_np)
    return pcd

# Konfiguracja strumienia
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

pipeline.start(config)

try:
    # Lista do przechowywania chmur punktów
    point_clouds = []

    for _ in range(3):  # Przetwarzanie 3 klatek
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()

        if not depth_frame or not color_frame:
            continue

        # Dodanie chmury punktów do listy
        point_clouds.append(frame_to_point_cloud(depth_frame))

    # Połączenie chmur punktów
    combined_pcd = point_clouds[0]
    for pcd in point_clouds[1:]:
        combined_pcd += pcd

    # Opcjonalnie: Możesz zastosować dalsze przetwarzanie, np. downsampling
    # combined_pcd = combined_pcd.voxel_down_sample(voxel_size=0.005)

    # Wyświetlenie połączonej chmury punktów
    o3d.visualization.draw_geometries([combined_pcd])
finally:
    pipeline.stop()
